# No good way to do metrics on this data other than to run the full test data set against it and calculate accuracy.

In [1]:
#imports
import os
from os.path import exists
import sys
import time
import numpy as np
from typing import Any, List, Tuple, Union
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
from skimage import io
from skimage import data
from skimage.util import compare_images
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import models
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetV2L, EfficientNetV2S, EfficientNetV2B0
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint
import pickle


tf.get_logger().setLevel('ERROR')           # Suppress TensorFlow logging (2)

#Use this to check if the GPU is configured correctly
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1715463740903251233
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5769199616
locality {
  bus_id: 1
  links {
  }
}
incarnation: 245920734475200933
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


In [2]:
#constants
PROJECT_DATA_PATH = './Data/'
#image expected size
#IMAGE_HEIGHT = 224 #b0
#IMAGE_WIDTH = 224 #b0
IMAGE_HEIGHT = 480 #l
IMAGE_WIDTH = 480 #l

modelname = "EfficientNetV2L"

In [3]:
image_size = (IMAGE_WIDTH, IMAGE_HEIGHT)
batch_size = 32

#test_ds = tf.keras.preprocessing.image_dataset_from_directory(
#    PROJECT_DATA_PATH + "test",
#    labels='inferred',
#    label_mode='categorical',
#    seed=1337,
#    image_size=image_size,
#    batch_size=batch_size,
#)

#num_classes = test_ds.element_spec[1].shape[1]
#print(f'Train has {num_classes} classes.')


labels = os.listdir(PROJECT_DATA_PATH + 'valid')

In [4]:
def load_model_data(model_path, opt_path):
    model = load_model(model_path)
    with open(opt_path, 'rb') as fp:
      d = pickle.load(fp)
      epoch = d['epoch']
      opt = d['opt']
      return epoch, model, opt

In [5]:
model_path = PROJECT_DATA_PATH + f'models/EfficientNetV2L/model-60-3.04.hdf5'
print(model_path)
opt_path = model_path.rsplit( ".", 1 )[ 0 ] 
opt_path += ".pkl"
print(opt_path)
epoch, model, opt = load_model_data(model_path, opt_path)

./Data/models/EfficientNetV2L/model-60-3.04.hdf5
./Data/models/EfficientNetV2L/model-60-3.04.pkl


In [6]:
model.compile(
    optimizer=tf.keras.optimizers.Adam.from_config(opt),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=["accuracy"],
)

In [7]:
def predict(img_array):
    predictions = model.predict(img_array)
    label = 'huh'
    idx = 0
    i = 0
    score = 0.0
    for x in predictions[0]:
        if x > score:
            score = x
            i = idx
        idx = idx + 1
    label = labels[i]
    #print(f"This image is {score*100} percent confidence is a(n) {label}.")
    return label, round(score*100, 2)

In [21]:
total = 1625
totalCorrect = 0
#epoch 45
for subdirs, dirs, files in os.walk(PROJECT_DATA_PATH + "test"):
    for subdir in dirs:
        #print(subdir) #this is the class name.
        imgCount = 0
        imgCorrect = 0
        for file in os.listdir(PROJECT_DATA_PATH + f"test/{subdir}"):
            imgCount = imgCount + 1
            img = keras.preprocessing.image.load_img(
                PROJECT_DATA_PATH + f"test/{subdir}/{file}", target_size=image_size
            )
            img_array = keras.preprocessing.image.img_to_array(img)
            img_array = tf.expand_dims(img_array, 0)  # Create batch axis
            species, score = predict(img_array)
            if species == subdir:
                imgCorrect = imgCorrect + 1
        print(f"{subdir} accuracy: {imgCorrect}/{imgCount} {round(imgCorrect/imgCount, 2)}\r\n")
        totalCorrect = totalCorrect + imgCorrect
        

print(f"Total accuracy: {totalCorrect}/{total} {round(totalCorrect/total, 2)}\r\n\r\n")  

AFRICAN CROWNED CRANE
AFRICAN CROWNED CRANE accuracy: 0/5 0.0


AFRICAN FIREFINCH
AFRICAN FIREFINCH accuracy: 4/5 0.8


ALBATROSS
ALBATROSS accuracy: 5/5 1.0


ALEXANDRINE PARAKEET
ALEXANDRINE PARAKEET accuracy: 4/5 0.8


AMERICAN AVOCET
AMERICAN AVOCET accuracy: 5/5 1.0


AMERICAN BITTERN
AMERICAN BITTERN accuracy: 4/5 0.8


AMERICAN COOT
AMERICAN COOT accuracy: 5/5 1.0


AMERICAN GOLDFINCH
AMERICAN GOLDFINCH accuracy: 5/5 1.0


AMERICAN KESTREL
AMERICAN KESTREL accuracy: 4/5 0.8


AMERICAN PIPIT
AMERICAN PIPIT accuracy: 5/5 1.0


AMERICAN REDSTART
AMERICAN REDSTART accuracy: 1/5 0.2


ANHINGA
ANHINGA accuracy: 5/5 1.0


ANNAS HUMMINGBIRD
ANNAS HUMMINGBIRD accuracy: 0/5 0.0


ANTBIRD
ANTBIRD accuracy: 1/5 0.2


ARARIPE MANAKIN
ARARIPE MANAKIN accuracy: 4/5 0.8


ASIAN CRESTED IBIS
ASIAN CRESTED IBIS accuracy: 5/5 1.0


BALD EAGLE
BALD EAGLE accuracy: 3/5 0.6


BALD IBIS
BALD IBIS accuracy: 3/5 0.6


BALI STARLING
BALI STARLING accuracy: 5/5 1.0


BALTIMORE ORIOLE
BALTIMORE ORIOLE accu

HOUSE FINCH accuracy: 5/5 1.0


HOUSE SPARROW
HOUSE SPARROW accuracy: 2/5 0.4


HYACINTH MACAW
HYACINTH MACAW accuracy: 4/5 0.8


IMPERIAL SHAQ
IMPERIAL SHAQ accuracy: 3/5 0.6


INCA TERN
INCA TERN accuracy: 3/5 0.6


INDIAN BUSTARD
INDIAN BUSTARD accuracy: 4/5 0.8


INDIAN PITTA
INDIAN PITTA accuracy: 2/5 0.4


INDIAN ROLLER
INDIAN ROLLER accuracy: 5/5 1.0


INDIGO BUNTING
INDIGO BUNTING accuracy: 0/5 0.0


IWI
IWI accuracy: 4/5 0.8


JABIRU
JABIRU accuracy: 5/5 1.0


JAVA SPARROW
JAVA SPARROW accuracy: 2/5 0.4


KAGU
KAGU accuracy: 5/5 1.0


KAKAPO
KAKAPO accuracy: 3/5 0.6


KILLDEAR
KILLDEAR accuracy: 4/5 0.8


KING VULTURE
KING VULTURE accuracy: 1/5 0.2


KIWI
KIWI accuracy: 4/5 0.8


KOOKABURRA
KOOKABURRA accuracy: 3/5 0.6


LARK BUNTING
LARK BUNTING accuracy: 3/5 0.6


LAZULI BUNTING
LAZULI BUNTING accuracy: 5/5 1.0


LILAC ROLLER
LILAC ROLLER accuracy: 2/5 0.4


LONG-EARED OWL
LONG-EARED OWL accuracy: 3/5 0.6


MAGPIE GOOSE
MAGPIE GOOSE accuracy: 5/5 1.0


MALABAR HORNBILL
MALAB

YELLOW HEADED BLACKBIRD accuracy: 3/5 0.6


Total accuracy: 1120/1625 0.69




In [ ]:
total = 1625
totalCorrect = 0
#epoch 43
for subdirs, dirs, files in os.walk(PROJECT_DATA_PATH + "test"):
    for subdir in dirs:
        #print(subdir) #this is the class name.
        imgCount = 0
        imgCorrect = 0
        for file in os.listdir(PROJECT_DATA_PATH + f"test/{subdir}"):
            imgCount = imgCount + 1
            img = keras.preprocessing.image.load_img(
                PROJECT_DATA_PATH + f"test/{subdir}/{file}", target_size=image_size
            )
            img_array = keras.preprocessing.image.img_to_array(img)
            img_array = tf.expand_dims(img_array, 0)  # Create batch axis
            species, score = predict(img_array)
            if species == subdir:
                imgCorrect = imgCorrect + 1
        print(f"{subdir} accuracy: {imgCorrect}/{imgCount} {round(imgCorrect/imgCount, 2)}\r\n")
        totalCorrect = totalCorrect + imgCorrect
        

print(f"Total accuracy: {totalCorrect}/{total} {round(totalCorrect/total, 2)}\r\n\r\n")    

AFRICAN CROWNED CRANE accuracy: 1/5 0.2

AFRICAN FIREFINCH accuracy: 5/5 1.0

ALBATROSS accuracy: 4/5 0.8

ALEXANDRINE PARAKEET accuracy: 4/5 0.8

AMERICAN AVOCET accuracy: 5/5 1.0

AMERICAN BITTERN accuracy: 5/5 1.0

AMERICAN COOT accuracy: 5/5 1.0

AMERICAN GOLDFINCH accuracy: 5/5 1.0

AMERICAN KESTREL accuracy: 4/5 0.8

AMERICAN PIPIT accuracy: 5/5 1.0

AMERICAN REDSTART accuracy: 3/5 0.6

ANHINGA accuracy: 5/5 1.0

ANNAS HUMMINGBIRD accuracy: 0/5 0.0

ANTBIRD accuracy: 3/5 0.6

ARARIPE MANAKIN accuracy: 5/5 1.0

ASIAN CRESTED IBIS accuracy: 5/5 1.0

BALD EAGLE accuracy: 5/5 1.0

BALD IBIS accuracy: 1/5 0.2

BALI STARLING accuracy: 5/5 1.0

BALTIMORE ORIOLE accuracy: 5/5 1.0

BANANAQUIT accuracy: 5/5 1.0

BANDED BROADBILL accuracy: 5/5 1.0

BANDED PITA accuracy: 5/5 1.0

BAR-TAILED GODWIT accuracy: 3/5 0.6

BARN OWL accuracy: 5/5 1.0

BARN SWALLOW accuracy: 5/5 1.0

BARRED PUFFBIRD accuracy: 4/5 0.8

BAY-BREASTED WARBLER accuracy: 4/5 0.8

BEARDED BARBET accuracy: 4/5 0.8

BEARDED B

In [12]:
total = 1625
totalCorrect = 0
#epoch 52
for subdirs, dirs, files in os.walk(PROJECT_DATA_PATH + "test"):
    for subdir in dirs:
        #print(subdir) #this is the class name.
        imgCount = 0
        imgCorrect = 0
        for file in os.listdir(PROJECT_DATA_PATH + f"test/{subdir}"):
            imgCount = imgCount + 1
            img = keras.preprocessing.image.load_img(
                PROJECT_DATA_PATH + f"test/{subdir}/{file}", target_size=image_size
            )
            img_array = keras.preprocessing.image.img_to_array(img)
            img_array = tf.expand_dims(img_array, 0)  # Create batch axis
            species, score = predict(img_array)
            if species == subdir:
                imgCorrect = imgCorrect + 1
        print(f"{subdir} accuracy: {imgCorrect}/{imgCount} {round(imgCorrect/imgCount, 2)}\r\n")
        totalCorrect = totalCorrect + imgCorrect
        

print(f"Total accuracy: {totalCorrect}/{total} {round(totalCorrect/total, 2)}\r\n\r\n")    

AFRICAN CROWNED CRANE accuracy: 1/5 0.2

AFRICAN FIREFINCH accuracy: 5/5 1.0

ALBATROSS accuracy: 5/5 1.0

ALEXANDRINE PARAKEET accuracy: 4/5 0.8

AMERICAN AVOCET accuracy: 5/5 1.0

AMERICAN BITTERN accuracy: 5/5 1.0

AMERICAN COOT accuracy: 5/5 1.0

AMERICAN GOLDFINCH accuracy: 5/5 1.0

AMERICAN KESTREL accuracy: 4/5 0.8

AMERICAN PIPIT accuracy: 5/5 1.0

AMERICAN REDSTART accuracy: 5/5 1.0

ANHINGA accuracy: 5/5 1.0

ANNAS HUMMINGBIRD accuracy: 1/5 0.2

ANTBIRD accuracy: 3/5 0.6

ARARIPE MANAKIN accuracy: 5/5 1.0

ASIAN CRESTED IBIS accuracy: 4/5 0.8

BALD EAGLE accuracy: 5/5 1.0

BALD IBIS accuracy: 4/5 0.8

BALI STARLING accuracy: 5/5 1.0

BALTIMORE ORIOLE accuracy: 5/5 1.0

BANANAQUIT accuracy: 5/5 1.0

BANDED BROADBILL accuracy: 5/5 1.0

BANDED PITA accuracy: 5/5 1.0

BAR-TAILED GODWIT accuracy: 5/5 1.0

BARN OWL accuracy: 5/5 1.0

BARN SWALLOW accuracy: 5/5 1.0

BARRED PUFFBIRD accuracy: 4/5 0.8

BAY-BREASTED WARBLER accuracy: 5/5 1.0

BEARDED BARBET accuracy: 4/5 0.8

BEARDED B

RED FACED WARBLER accuracy: 5/5 1.0

RED FODY accuracy: 5/5 1.0

RED HEADED DUCK accuracy: 5/5 1.0

RED HEADED WOODPECKER accuracy: 5/5 1.0

RED HONEY CREEPER accuracy: 3/5 0.6

RED NAPED TROGON accuracy: 3/5 0.6

RED TAILED HAWK accuracy: 4/5 0.8

RED TAILED THRUSH accuracy: 5/5 1.0

RED WINGED BLACKBIRD accuracy: 4/5 0.8

RED WISKERED BULBUL accuracy: 5/5 1.0

REGENT BOWERBIRD accuracy: 5/5 1.0

RING-NECKED PHEASANT accuracy: 5/5 1.0

ROADRUNNER accuracy: 5/5 1.0

ROBIN accuracy: 5/5 1.0

ROCK DOVE accuracy: 4/5 0.8

ROSY FACED LOVEBIRD accuracy: 2/5 0.4

ROUGH LEG BUZZARD accuracy: 0/5 0.0

ROYAL FLYCATCHER accuracy: 5/5 1.0

RUBY THROATED HUMMINGBIRD accuracy: 3/5 0.6

RUDY KINGFISHER accuracy: 4/5 0.8

RUFOUS KINGFISHER accuracy: 2/5 0.4

RUFUOS MOTMOT accuracy: 3/5 0.6

SAMATRAN THRUSH accuracy: 5/5 1.0

SAND MARTIN accuracy: 5/5 1.0

SANDHILL CRANE accuracy: 4/5 0.8

SATYR TRAGOPAN accuracy: 2/5 0.4

SCARLET CROWNED FRUIT DOVE accuracy: 5/5 1.0

SCARLET IBIS accuracy: 5/5 1.0

S

In [29]:
total = 1625
totalCorrect = 0
#epoch 59
for subdirs, dirs, files in os.walk(PROJECT_DATA_PATH + "test"):
    for subdir in dirs:
        #print(subdir) #this is the class name.
        imgCount = 0
        imgCorrect = 0
        for file in os.listdir(PROJECT_DATA_PATH + f"test/{subdir}"):
            imgCount = imgCount + 1
            img = keras.preprocessing.image.load_img(
                PROJECT_DATA_PATH + f"test/{subdir}/{file}", target_size=image_size
            )
            img_array = keras.preprocessing.image.img_to_array(img)
            img_array = tf.expand_dims(img_array, 0)  # Create batch axis
            species, score = predict(img_array)
            if species == subdir:
                imgCorrect = imgCorrect + 1
        print(f"{subdir} accuracy: {imgCorrect}/{imgCount} {round(imgCorrect/imgCount, 2)}\r\n")
        totalCorrect = totalCorrect + imgCorrect
        

print(f"Total accuracy: {totalCorrect}/{total} {round(totalCorrect/total, 2)}\r\n\r\n")    

AFRICAN CROWNED CRANE accuracy: 0/5 0.0

AFRICAN FIREFINCH accuracy: 2/5 0.4

ALBATROSS accuracy: 3/5 0.6

ALEXANDRINE PARAKEET accuracy: 5/5 1.0

AMERICAN AVOCET accuracy: 5/5 1.0

AMERICAN BITTERN accuracy: 4/5 0.8

AMERICAN COOT accuracy: 5/5 1.0

AMERICAN GOLDFINCH accuracy: 5/5 1.0

AMERICAN KESTREL accuracy: 4/5 0.8

AMERICAN PIPIT accuracy: 2/5 0.4

AMERICAN REDSTART accuracy: 1/5 0.2

ANHINGA accuracy: 3/5 0.6

ANNAS HUMMINGBIRD accuracy: 0/5 0.0

ANTBIRD accuracy: 2/5 0.4

ARARIPE MANAKIN accuracy: 4/5 0.8

ASIAN CRESTED IBIS accuracy: 4/5 0.8

BALD EAGLE accuracy: 4/5 0.8

BALD IBIS accuracy: 2/5 0.4

BALI STARLING accuracy: 5/5 1.0

BALTIMORE ORIOLE accuracy: 3/5 0.6

BANANAQUIT accuracy: 2/5 0.4

BANDED BROADBILL accuracy: 5/5 1.0

BANDED PITA accuracy: 4/5 0.8

BAR-TAILED GODWIT accuracy: 3/5 0.6

BARN OWL accuracy: 4/5 0.8

BARN SWALLOW accuracy: 2/5 0.4

BARRED PUFFBIRD accuracy: 3/5 0.6

BAY-BREASTED WARBLER accuracy: 3/5 0.6

BEARDED BARBET accuracy: 4/5 0.8

BEARDED B

RED FACED WARBLER accuracy: 5/5 1.0

RED FODY accuracy: 1/5 0.2

RED HEADED DUCK accuracy: 4/5 0.8

RED HEADED WOODPECKER accuracy: 4/5 0.8

RED HONEY CREEPER accuracy: 2/5 0.4

RED NAPED TROGON accuracy: 3/5 0.6

RED TAILED HAWK accuracy: 0/5 0.0

RED TAILED THRUSH accuracy: 5/5 1.0

RED WINGED BLACKBIRD accuracy: 3/5 0.6

RED WISKERED BULBUL accuracy: 5/5 1.0

REGENT BOWERBIRD accuracy: 3/5 0.6

RING-NECKED PHEASANT accuracy: 4/5 0.8

ROADRUNNER accuracy: 5/5 1.0

ROBIN accuracy: 5/5 1.0

ROCK DOVE accuracy: 4/5 0.8

ROSY FACED LOVEBIRD accuracy: 2/5 0.4

ROUGH LEG BUZZARD accuracy: 0/5 0.0

ROYAL FLYCATCHER accuracy: 5/5 1.0

RUBY THROATED HUMMINGBIRD accuracy: 5/5 1.0

RUDY KINGFISHER accuracy: 0/5 0.0

RUFOUS KINGFISHER accuracy: 2/5 0.4

RUFUOS MOTMOT accuracy: 1/5 0.2

SAMATRAN THRUSH accuracy: 5/5 1.0

SAND MARTIN accuracy: 4/5 0.8

SANDHILL CRANE accuracy: 5/5 1.0

SATYR TRAGOPAN accuracy: 5/5 1.0

SCARLET CROWNED FRUIT DOVE accuracy: 5/5 1.0

SCARLET IBIS accuracy: 4/5 0.8

S

In [8]:
total = 1625
totalCorrect = 0
#epoch 60 with resized images
for subdirs, dirs, files in os.walk(PROJECT_DATA_PATH + "test"):
    for subdir in dirs:
        #print(subdir) #this is the class name.
        imgCount = 0
        imgCorrect = 0
        for file in os.listdir(PROJECT_DATA_PATH + f"test/{subdir}"):
            imgCount = imgCount + 1
            img = keras.preprocessing.image.load_img(
                PROJECT_DATA_PATH + f"test/{subdir}/{file}", target_size=image_size
            )
            img_array = keras.preprocessing.image.img_to_array(img)
            img_array = tf.expand_dims(img_array, 0)  # Create batch axis
            species, score = predict(img_array)
            if species == subdir:
                imgCorrect = imgCorrect + 1
        print(f"{subdir} accuracy: {imgCorrect}/{imgCount} {round(imgCorrect/imgCount, 2)}")
        totalCorrect = totalCorrect + imgCorrect
        

print(f"Total accuracy: {totalCorrect}/{total} {round(totalCorrect/total, 2)}\r\n\r\n")    

AFRICAN CROWNED CRANE accuracy: 0/5 0.0

AFRICAN FIREFINCH accuracy: 5/5 1.0

ALBATROSS accuracy: 5/5 1.0

ALEXANDRINE PARAKEET accuracy: 4/5 0.8

AMERICAN AVOCET accuracy: 5/5 1.0

AMERICAN BITTERN accuracy: 5/5 1.0

AMERICAN COOT accuracy: 5/5 1.0

AMERICAN GOLDFINCH accuracy: 5/5 1.0

AMERICAN KESTREL accuracy: 4/5 0.8

AMERICAN PIPIT accuracy: 5/5 1.0

AMERICAN REDSTART accuracy: 3/5 0.6

ANHINGA accuracy: 5/5 1.0

ANNAS HUMMINGBIRD accuracy: 0/5 0.0

ANTBIRD accuracy: 2/5 0.4

ARARIPE MANAKIN accuracy: 5/5 1.0

ASIAN CRESTED IBIS accuracy: 3/5 0.6

BALD EAGLE accuracy: 5/5 1.0

BALD IBIS accuracy: 0/5 0.0

BALI STARLING accuracy: 5/5 1.0

BALTIMORE ORIOLE accuracy: 5/5 1.0

BANANAQUIT accuracy: 5/5 1.0

BANDED BROADBILL accuracy: 5/5 1.0

BANDED PITA accuracy: 5/5 1.0

BAR-TAILED GODWIT accuracy: 5/5 1.0

BARN OWL accuracy: 5/5 1.0

BARN SWALLOW accuracy: 5/5 1.0

BARRED PUFFBIRD accuracy: 3/5 0.6

BAY-BREASTED WARBLER accuracy: 4/5 0.8

BEARDED BARBET accuracy: 4/5 0.8

BEARDED B

RED FACED WARBLER accuracy: 5/5 1.0

RED FODY accuracy: 4/5 0.8

RED HEADED DUCK accuracy: 0/5 0.0

RED HEADED WOODPECKER accuracy: 5/5 1.0

RED HONEY CREEPER accuracy: 2/5 0.4

RED NAPED TROGON accuracy: 3/5 0.6

RED TAILED HAWK accuracy: 4/5 0.8

RED TAILED THRUSH accuracy: 5/5 1.0

RED WINGED BLACKBIRD accuracy: 5/5 1.0

RED WISKERED BULBUL accuracy: 5/5 1.0

REGENT BOWERBIRD accuracy: 5/5 1.0

RING-NECKED PHEASANT accuracy: 4/5 0.8

ROADRUNNER accuracy: 5/5 1.0

ROBIN accuracy: 5/5 1.0

ROCK DOVE accuracy: 5/5 1.0

ROSY FACED LOVEBIRD accuracy: 5/5 1.0

ROUGH LEG BUZZARD accuracy: 0/5 0.0

ROYAL FLYCATCHER accuracy: 5/5 1.0

RUBY THROATED HUMMINGBIRD accuracy: 5/5 1.0

RUDY KINGFISHER accuracy: 3/5 0.6

RUFOUS KINGFISHER accuracy: 4/5 0.8

RUFUOS MOTMOT accuracy: 4/5 0.8

SAMATRAN THRUSH accuracy: 4/5 0.8

SAND MARTIN accuracy: 1/5 0.2

SANDHILL CRANE accuracy: 5/5 1.0

SATYR TRAGOPAN accuracy: 3/5 0.6

SCARLET CROWNED FRUIT DOVE accuracy: 5/5 1.0

SCARLET IBIS accuracy: 5/5 1.0

S